In [ ]:
!pip install text_normalizer
import pandas as pd
import numpy as np
import text_normalizer as tn

In [2]:
from google.colab import userdata
import os
os.environ["KAGGLE_KEY"] = userdata.get("KAAGLE_KEY")
os.environ["KAGGLE_USERNAME"] = userdata.get("KAAGLE_USERNAME")

In [3]:
!kaggle datasets download -d ashfakyeafi/Fake_message_classification

  0% 0.00/207k [00:00<?, ?B/s]
100% 207k/207k [00:00<00:00, 83.4MB/s]


In [4]:
!unzip "Fake_message_classification.zip"

Archive:  spam-email-classification.zip
  inflating: email.csv               


In [5]:
data_df = pd.read_csv("/content/email.csv")
data_df

Category                                            Message
0                ham  Go until jurong point, crazy.. Available only ...
1                ham                      Ok lar... Joking wif u oni...
2               spam  Free entry in 2 a wkly comp to win FA Cup fina...
3                ham  U dun say so early hor... U c already then say...
4                ham  Nah I don't think he goes to usf, he lives aro...
...              ...                                                ...
5568             ham               Will ü b going to esplanade fr home?
5569             ham  Pity, * was in mood for that. So...any other s...
5570             ham  The guy did some bitching but I acted like i'd...
5571             ham                         Rofl. Its true to its name
5572  {"mode":"full"                                    isActive:false}

[5573 rows x 2 columns]

In [6]:
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names =\
                                 train_test_split(np.array(data_df['Message']), np.array(data_df['Category']),
                                                       np.array(data_df['Category']), test_size=0.33, random_state=42)

train_corpus.shape, test_corpus.shape

((3733,), (1840,))

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokenized_train = [word_tokenize(text) for text in train_corpus]
tokenized_test = [word_tokenize(text) for text in test_corpus]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import gensim
import gensim


w2v_num_features = 1000
w2v_model = gensim.models.Word2Vec(tokenized_train, vector_size=w2v_num_features, window=100,
                                   min_count=2, sample=1e-3, sg=1, epochs=5, workers=10)

In [9]:
def document_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)

    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.

        for word in words:
            if word in vocabulary:
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [10]:

def document_vectorizer(corpus, model, num_features):

    features = np.zeros((len(corpus), num_features), dtype="float32")

    for i, tokens in enumerate(corpus):

        valid_tokens = [token for token in tokens if token in model.wv.key_to_index]
        if valid_tokens:
            features[i] = np.mean([model.wv[token] for token in valid_tokens], axis=0)

    return features

avg_wv_train_features = document_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = document_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)



In [11]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape,
      ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (3733, 1000)  Test features shape: (1840, 1000)


In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(loss='hinge', penalty='l2', random_state=42, max_iter=500)
svm.fit(avg_wv_train_features, train_label_names)
svm_w2v_cv_scores = cross_val_score(svm, avg_wv_train_features, train_label_names, cv=2)
svm_w2v_cv_mean_score = np.mean(svm_w2v_cv_scores)
print('CV Accuracy (5-fold):', svm_w2v_cv_scores)
print('Mean CV Accuracy:', svm_w2v_cv_mean_score)
svm_w2v_test_score = svm.score(avg_wv_test_features, test_label_names)
print('Test Accuracy:', svm_w2v_test_score)

CV Accuracy (5-fold): [0.97429031 0.9624866 ]
Mean CV Accuracy: 0.9683884538303047
Test Accuracy: 0.967391304347826
